In [5]:
! pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=95a34a0fe8401e2e5f9631da78df32749eeab39b5dd69706056eb49516d986d3
  Stored in directory: c:\users\vasuki.k\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built termcolor


In [4]:
# Standard Imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import log_loss

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense, Dropout, Activation

from hyperopt import tpe, STATUS_OK, Trials
from hyperas import optim
from hyperas.distributions import choice, uniform

ModuleNotFoundError: No module named 'tensorflow'

## Let's create a function which processes the data in such a way that that the data is directly fed into the model

In [36]:
def data():  
  global test
  # Importing data
  train = pd.read_csv("/content/drive/Othercomputers/My Laptop (1)/Programming Languages/Python/Projects/loan-defaulter/data/train.csv")
  test = pd.read_csv("/content/drive/Othercomputers/My Laptop (1)/Programming Languages/Python/Projects/loan-defaulter/data/test.csv")

  # Dropping unnecessary columns in both test and training sets
  train = train.drop("ID", axis=1)
  test = test.drop(["Loan Status", "ID"], axis=1)

  # Let's split the data into features and target variables
  X = train.drop("Loan Status", axis=1)
  y = train["Loan Status"]

  # Splitting the data into training and validation sets
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

  # Converting non- numerical columns into numerica Columns
  for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
      encoder = LabelEncoder()
      X_train[label] = encoder.fit_transform(X_train[label])
      X_valid[label] = encoder.transform(X_valid[label])
      test[label] = encoder.transform(test[label])

  # Now we scale the data
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_valid = scaler.transform(X_valid)
  test = scaler.transform(test)

  return X_train, y_train, X_valid, y_valid

# Calling our function
X_train, y_train, X_valid, y_valid = data()

## Let's create our model

In [4]:
def model():
  # Set random seed
  tf.random.set_seed(42)

  # Create a model
  model = Sequential()
  model.add(Input(shape=X_train[0].shape))
  model.add(Dense({{choice([20, 30, 40])}}))
  model.add(Activation("relu"))
  model.add(Dropout({{uniform(0, 1)}}))
  model.add(Dense({{choice([10, 20, 30])}}))
  model.add(Activation("relu"))
  model.add(Dropout({{uniform(0, 1)}}))

  # Let's add another layer according to choice
  if {{choice(["a", "b"])}} == "b":
    model.add(Dense(5, activation="relu"))
    model.add(Dropout({{uniform(0, 1)}}))

  # Let's add the final layer
  model.add(Dense(1))
  model.add(Activation({{choice(["sigmoid", "softmax"])}}))

  # let's compile the model
  model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate={{choice([0.1, 0.001, 0.01, 3e-4])}}),
                metrics=["accuracy"])
  
  # Fit the model
  model.fit(X_train, y_train,
            validation_data=(X_valid, y_valid),
            epochs=100,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)], verbose=0)
  
  # Evaluate our model
  loss, acc = model.evaluate(X_valid, y_valid, verbose=0)
  print("Validation accuracy :", acc)

  return {"loss" : -acc, "model" : model, 'status' : STATUS_OK}

In [5]:
# Now let's optimize our model
best_run, best_model = optim.minimize(data=data,
                                      model=model,
                                      max_evals=10,
                                      algo=tpe.suggest,
                                      trials=Trials(),
                                      notebook_name="drive/Othercomputers/My Laptop (1)/Programming Languages/Python/Projects/loan-defaulter/loan_defaulter-hyperas-colab")

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler, LabelEncoder
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Dense, Dropout, Activation
except:
    pass

try:
    from hyperopt import tpe, STATUS_OK, Trials
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [20, 30, 40]),
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense_1': hp.choice('Dense_1', [10, 20, 30]),
        'Dropout_1': hp.uniform('Dropout_1', 0, 

In [9]:
log_loss(y_valid, best_model.predict(X_valid))

0.3034542779984374

In [13]:
best_run

{'Activation': 0,
 'Dense': 2,
 'Dense_1': 2,
 'Dropout': 0.6108763092812357,
 'Dropout_1': 0.7371698374615214,
 'Dropout_2': 0,
 'Dropout_3': 0.21280043312755825,
 'learning_rate': 2}

In [45]:
# Final model

# Create a model
final = Sequential([
  Input(shape=X_train[0].shape),
  Dense(100, activation="relu"),
  Dropout(0.6108763092812357),
  Dense(50, activation="relu"),
  Dropout(0.7371698374615214),
  Dense(25, activation="relu"),
  Dropout(0.21280043312755825),
  Dense(1, activation="sigmoid")
])

# Compile the model
final.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])

#Fit the model
history =   final.fit(X_train, y_train, epochs=100,
            validation_data=(X_valid, y_valid),
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3989 - accuracy: 0.8834 - val_loss: 0.3436 - val_accuracy: 0.9097
Epoch 2/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3445 - accuracy: 0.9068 - val_loss: 0.3312 - val_accuracy: 0.9097
Epoch 3/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3264 - accuracy: 0.9069 - val_loss: 0.3145 - val_accuracy: 0.9097
Epoch 4/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3189 - accuracy: 0.9069 - val_loss: 0.3061 - val_accuracy: 0.9097
Epoch 5/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3157 - accuracy: 0.9069 - val_loss: 0.3038 - val_accuracy: 0.9097
Epoch 6/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3145 - accuracy: 0.9069 - val_loss: 0.3035 - val_accuracy: 0.9097
Epoch 7/100
1687/1687 [==============================] - 5s 3ms/step - loss: 0.3130 - accuracy: 0.9069 - val_loss: 0.3035 - val_ac

In [46]:
predictions=final.predict(test).flatten()
submissions = pd.DataFrame()
predictions

array([0.0990999 , 0.08767685, 0.0990999 , ..., 0.09805545, 0.09900853,
       0.09895027], dtype=float32)

In [47]:
log_loss(y_valid, final.predict(X_valid))

0.3034117250054255

In [48]:
submissions["Loan Status"] = predictions
submissions.to_csv("predictions.csv", index=False)